In [4]:
import matplotlib.pyplot as plt
from src.FunctionsNetworkPlot import *

In [2]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import networkx as nx
import numpy as np
import os

# === Funções auxiliares ===
def calculate_center_of_mass(pos):
    xs = [x for x, _ in pos.values()]
    ys = [y for _, y in pos.values()]
    return np.mean(xs), np.mean(ys)

def get_limits_for_first_subplot(pos):
    xs = [x for x, _ in pos.values()]
    ys = [y for _, y in pos.values()]
    margin = 0.1
    xrange = max(xs) - min(xs)
    yrange = max(ys) - min(ys)
    return (min(xs) - margin*xrange, max(xs) + margin*xrange), (min(ys) - margin*yrange, max(ys) + margin*yrange)

def draw_graph_image(G, pos, alpha_a, alpha_g, output_dir):
    fig, ax = plt.subplots(figsize=(6, 6))

    cmap = plt.get_cmap('viridis', len(pos))
    norm = mcolors.Normalize(vmin=0, vmax=len(pos) - 1)
    node_colors = [cmap(norm(i)) for i in range(len(pos))]

    node_list = sorted(pos.keys(), key=lambda x: int(x.split('_')[1]))
    x_cm, y_cm = calculate_center_of_mass(pos)
    G.add_node("cm", pos=(x_cm, y_cm))
    pos["cm"] = (x_cm, y_cm)

    nx.draw_networkx_edges(G, pos, ax=ax, edge_color='#000000', alpha=0.6, width=0.8)
    nx.draw_networkx_nodes(G, pos, ax=ax, nodelist=node_list, node_size=120,
                           node_color=node_colors, edgecolors='#000000', linewidths=0.8)
    nx.draw_networkx_nodes(G, pos, ax=ax, nodelist=["cm"], node_size=120,
                           node_color="red", edgecolors="black", linewidths=2.0)
    
    global_x_limits, global_y_limits = get_limits_for_first_subplot(pos)
    ax.set_xlim(global_x_limits)
    ax.set_ylim(global_y_limits)

    ax.set_xticks(np.linspace(*ax.get_xlim(), num=3))
    ax.set_yticks(np.linspace(*ax.get_ylim(), num=4))
    ax.set_xlabel(r"$X$", fontsize=30)
    ax.set_ylabel(r"$Y$", fontsize=30)

    ax.tick_params(axis='both', which='both', length=4.0, width=2.0,
                   direction='in', labelsize=20, pad=10.0,
                   labelbottom=True, labelleft=True)

    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_visible(True)

    for spine in ax.spines.values():
        spine.set_linewidth(2.0)

    ax.grid(False)
    plt.tight_layout()

    os.makedirs(output_dir, exist_ok=True)

    # Substituir "." por "_" nos nomes dos parâmetros
    alpha_a_str = str(alpha_a).replace('.', '_')
    alpha_g_str = str(alpha_g).replace('.', '_')
    filename = os.path.join(output_dir, f"alpha_a_{alpha_a_str}_alpha_g_{alpha_g_str}.png")

    plt.savefig(filename, dpi=300)
    plt.close()


In [3]:
# === Parâmetros do modelo ===
N = 500
alpha_ag_f = 2.0
alpha_g_v = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]
dim = 2
output_dir = "images_networks_alphaG"

# === Geração das imagens ===
for alpha_g in alpha_g_v:
    G = nx.Graph()
    fn = select_first_gml_gz_file(N, dim, alpha_ag_f, alpha_g)
    edges, nodes_positions = positions_GML(N, dim, alpha_ag_f, alpha_g, fn)

    for node, pos_ in nodes_positions.items():
        G.add_node(node, pos=(pos_[0], pos_[1]))  # 2D somente

    G.add_edges_from(edges)
    pos = nx.get_node_attributes(G, 'pos')

    draw_graph_image(G, pos, alpha_ag_f, alpha_g, output_dir)

In [20]:
# === Parâmetros do modelo ===
N = 500
alpha_ag_f = 2.0
alpha_a_v = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
dim = 2
output_dir = "images_networks_alphaA"

# === Geração das imagens ===
for alpha_a in alpha_a_v:
    G = nx.Graph()
    fn = select_first_gml_gz_file(N, dim, alpha_a, alpha_ag_f)
    edges, nodes_positions = positions_GML(N, dim, alpha_a, alpha_ag_f, fn)

    for node, pos_ in nodes_positions.items():
        G.add_node(node, pos=(pos_[0], pos_[1]))  # 2D somente

    G.add_edges_from(edges)
    pos = nx.get_node_attributes(G, 'pos')

    draw_graph_image(G, pos, alpha_a, alpha_ag_f, output_dir)